In [1]:
# before

import IPython.display as ipd
ipd.Audio('../stress-test/test.mp3')

In [2]:
import requests

headers = {
    'accept': 'application/json',
}

params = {
    "file_response": "false",
    "response_format": "mp3"
}

files = {
    'file': ('audio.mp3', open('../stress-test/test.mp3', 'rb'), 'audio/mpeg'),
}

response = requests.post('http://localhost:7088/speech_enhancement', headers=headers, files=files, params=params)

In [3]:
import base64
import io
import soundfile as sf

In [4]:
r = response.json()
r.keys(), r['stats']

(dict_keys(['audio', 'stats']),
 {'total_length': 5.258956916099773, 'seconds_per_second': 6.345566200103549})

In [5]:
audio_binary = base64.b64decode(r['audio'])
buffer = io.BytesIO(audio_binary)
audio_array, sample_rate = sf.read(buffer)
len(audio_array) / sample_rate, sample_rate

(5.258956916099773, 44100)

In [6]:
ipd.Audio(audio_array, rate = sample_rate)

In [7]:
import aiohttp

async def process(file_path):
    url = "http://localhost:7088/speech_enhancement"
    
    async with aiohttp.ClientSession() as session:
        with open(file_path, "rb") as file:
            form_data = aiohttp.FormData()
            form_data.add_field(
                "file",
                file,
                filename="audio.mp3",
                content_type="audio/mpeg"
            )
            
            headers = {
                "accept": "application/json"
            }
            
            params = {
                "file_response": "false",
                "response_format": "mp3"
            }
            
            async with session.post(url, headers=headers, data=form_data, params=params) as response:
                if response.status == 200:
                    data = await response.json()
                    return data
                else:
                    print(f"Failed with status: {response.status}")
                    print("Response:", await response.text())

In [8]:
from glob import glob
import asyncio

files = glob('../stress-test/*.mp3')
files

['../stress-test/test.mp3', '../stress-test/anwar-ibrahim.mp3']

In [9]:
futures = [process(f) for f in files]
results = await asyncio.gather(*futures)

In [10]:
results[0]['stats']

{'total_length': 5.258956916099773, 'seconds_per_second': 1.8406906332575375}

In [11]:
results[1]['stats']

{'total_length': 14.631020408163264, 'seconds_per_second': 8.277038367207327}